This is my first notebook on Kaggle. Please leave your sugestions below.
Please upvote it if you find it useful and to show your support too. 
Thanks. 👍

I plan to write a code explanation for this and train the model further to improve the accuracy of the model. 

I have used the latest Fastai(Fastai version 2 library) to train the model and make predictions on test set.

In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch
print(torch.__version__)

In [ ]:
import fastai
print(fastai.__version__)

In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path('../input/input/siim-isic-melanoma-classification/')


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
print("Training samples", train_df.shape)
print("First few samples of data are",train_df.head())

In [ ]:
train_df.target.value_counts()


In [ ]:
train_df_output_1 = train_df[train_df['target']==1]
train_df.shape


In [ ]:
train_df_output_0 = train_df[train_df['target']==0].sample(frac=0.03)
train_df_output_0.shape


In [ ]:
new_df = pd.concat([train_df_output_0,train_df_output_1]).reset_index(drop=True)
new_df.shape


In [ ]:
new_df.head()
new_df.shape, 
new_df.target.value_counts()


In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)


In [ ]:
fold = 0
for train_index, test_index in skf.split(X=new_df.values, y=new_df.target.values):
    fold+= 1
    print("Fold",fold)
    print("TRAIN LENGTH:", len(train_index), "VALIDATION LENGTH:", len(test_index))
    new_df[f'fold_{fold}_valid']= 0
    new_df.loc[test_index,f'fold_{fold}_valid']= 1


In [ ]:
imgpath = Path('../input/siim-isic-melanoma-classification/jpeg/train')


In [ ]:
dls = ImageDataLoaders.from_df(new_df, path=imgpath,
                               seed=42, fn_col=0, 
                               suff='.jpg', label_col=7, 
                               valid_col=f'fold_1_valid', item_tfms=Resize(128), 
                               batch_tfms=aug_transforms(flip_vert=True, max_warp=0.), 
                               bs=128, val_bs=None, shuffle_train=True)


In [ ]:
print(torch.cuda.is_available())

In [ ]:
dls.device

In [ ]:
dls.show_batch()


In [ ]:
#learn = cnn_learner(dls,resnet34,metrics = [accuracy,roc_auc])
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)


In [ ]:
samplesubmit = "../input/siim-isic-melanoma-classification/sample_submission.csv"
sub = pd.read_csv(samplesubmit)
sub.head()


In [ ]:
for i in range(10982):
    x = sub.at[i,'image_name']
    imggpath = Path("../input/siim-isic-melanoma-classification/jpeg/test/" + x + ".jpg")
    pr1,_,pr2 = learn.predict(imggpath)
    sub.at[i,'target'] = float(pr2[int(pr1)])

In [ ]:
sub.head()

In [ ]:
sub.to_csv('sub1.csv', index=False)